In [1]:
import ee
import geemap
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sn
import time
import os
import re
from skimage.filters import try_all_threshold, threshold_otsu
from skimage.restoration import denoise_wavelet
from pkg import pkg
from pkg import save
from osgeo import gdal
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import spenv

In [2]:
ee.Authenticate()
ee.Initialize()
coordenadas = "-51.881964,3.928400,-51.746512,3.791524"
x1, y1, x2, y2 = coordenadas.split(",")

datas = "2018-01-01,2020-12-31"
inicio, fim = datas.split(",")
escala = 30
dummy_value = 99999

geom = ee.Geometry.Polygon([[[float(x1),float(y2)],
                             [float(x2),float(y2)],
                             [float(x2),float(y1)],
                             [float(x1),float(y1)],
                             [float(x1),float(y2)]]])


sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filterBounds(geom)\
    .filterDate(inicio,fim)\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    
v_emit_asc = sentinel1.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
v_emit_desc = sentinel1.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

image = ee.Image(dummy_value).blend(v_emit_desc.map(pkg.add_amplitude).select('amplitude').toBands())
image_names = image.bandNames().getInfo()

In [3]:
directory = 'testing'
files = [directory + "/" + f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and f.startswith('optimal')]

def extract_number(s):
    match = re.search(r'\d+', s)
    return int(match.group()) if match else float('inf')

sorted_files = sorted(files, key=extract_number)

Y = []
for n, i in enumerate(sorted_files):
    temp = rasterio.open(i)
    Y.append(temp.read())
    temp.close()
    print(str(n+1)+"/"+str(len(sorted_files))+": "+sorted_files[n], end="\r")

Y = np.concatenate(Y)

In [4]:
directory = 'testing'
files = [directory + "/" + f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and f.startswith('raster')]

def extract_number(s):
    match = re.search(r'\d+', s)
    return int(match.group()) if match else float('inf')

sorted_files = sorted(files, key=extract_number)

X = []
for n, i in enumerate(sorted_files):
    temp = rasterio.open(i)
    X.append(temp.read())
    temp.close()
    print(str(n+1)+"/"+str(len(sorted_files))+": "+sorted_files[n], end="\r")

X = np.concatenate(X)

In [5]:
del temp

In [6]:
std = np.std(X)
mean = np.mean(X)
Y_ = (Y * std) + mean

In [16]:
def ECS(x, smooth_x=None):
    
    assert isinstance(x, np.ndarray), "'x' is not a numpy.ndarray"
    assert len(x.shape) == 3, "'x' is not three-dimensional"

    mean_image = x.mean(axis=0)
    
    if smooth_x is not None:
        assert isinstance(smooth_x, np.ndarray), "'smooth_x' is not a numpy.ndarray"
        assert len(smooth_x.shape) == 3, "'smooth_x' is not three-dimensional"
        assert x.shape == smooth_x.shape, "'x' and 'smooth_x' are different shapes"
        cube = smooth_x
    else:
        cube = x

    R = np.ndarray(mean_image.shape)
    D = np.ndarray(cube.shape)

    dims = mean_image.shape
    
    lin = dims[0]
    col = dims[1]
    
    for i in range(0, cube.shape[0]):
        D[i] = (cube[i] - mean_image)**2
    
    d = D.sum(axis=(1, 2)).ravel()

    for i in range(lin):
        for j in range(col):
            R[i, j] = np.abs(np.corrcoef(d, D[:, i, j])[0][1])
    
    return R

In [ ]:
wav = denoise_wavelet(Y_[0], rescale_sigma=True)

fig, ax = plt.subplots(1, 2)
ax[0].imshow(Y_[1], cmap="gray")
ax[1].imshow(wav, cmap="gray")
plt.tight_layout()
plt.show()

In [ ]:
list(X)